# Retrieve generation data from AEMO

In [1]:
import nemosis, pandas as pd, numpy
from nemosis import static_table, dynamic_data_compiler
from pathlib import Path

In [3]:
raw_NEM_cache = "/scratch/w42/dr6273/raw_AEMO_cache"

In [4]:
# print(nemosis.defaults.static_tables)

['ELEMENTS_FCAS_4_SECOND', 'VARIABLES_FCAS_4_SECOND', 'Generators and Scheduled Loads', 'FCAS Providers']


In [5]:
# # Downloading the Generators and Scheduled Loads table from NEMOSIS
# NEM_gen = static_table(
#     table_name="Generators and Scheduled Loads", 
#     raw_data_location=raw_NEM_cache,
#     update_static_file=False
# )
                              
# #  Selecting only generating units, not loads or batteries
# NEM_gen = NEM_gen.loc[NEM_gen['Dispatch Type'] == "Generating Unit"]

INFO: Retrieving static table Generators and Scheduled Loads


In [9]:
# NEM_gen.head()

In [8]:
# Downloading the Generator and Load Dispatch table from NEMOSIS for a specified time period
dynamic_data_compiler(
    start_time='2024/01/01 00:00:00',
    end_time='2024/01/01 00:10:00',
    table_name='DISPATCHLOAD',
    raw_data_location=raw_NEM_cache
)

INFO: Compiling data for table DISPATCHLOAD
INFO: Returning DISPATCHLOAD.


,SETTLEMENTDATE,DUID,INTERVENTION,DISPATCHMODE,AGCSTATUS,INITIALMW,TOTALCLEARED,RAMPDOWNRATE,RAMPUPRATE,LOWER5MIN,...,LOWERREG,RAISEREG,AVAILABILITY,RAISEREGENABLEMENTMAX,RAISEREGENABLEMENTMIN,LOWERREGENABLEMENTMAX,LOWERREGENABLEMENTMIN,SEMIDISPATCHCAP,LOWER1SEC,RAISE1SEC
0,2024-01-01 00:05:00,ADPBA1G,0,0,1,5.94300,2.00000,93.12,93.12,0.0,...,0.0,0,6.00000,6.0,0.0,6.0,0.0,0,0.0,0
1,2024-01-01 00:05:00,ADPBA1L,0,0,1,0.00000,0.00000,93.12,93.12,3.0,...,0.0,0,6.00000,6.0,0.0,6.0,0.0,0,0.0,0
2,2024-01-01 00:05:00,ADPPV1,0,0,0,0.00000,0.00000,120.00,120.00,0.0,...,0.0,0,0.00000,0.0,0.0,0.0,0.0,0,0.0,0
3,2024-01-01 00:05:00,AGLHAL,0,0,0,0.00000,0.00000,720.00,720.00,0.0,...,0.0,0,200.00000,0.0,0.0,0.0,0.0,0,0.0,0
4,2024-01-01 00:05:00,AGLSOM,0,0,0,0.00000,0.00000,480.00,480.00,0.0,...,0.0,0,170.00000,0.0,0.0,0.0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
955,2024-01-01 00:10:00,YENDWF1,0,0,0,39.65000,38.14476,1680.00,1680.00,0.0,...,0.0,0,38.14476,0.0,0.0,0.0,0.0,0,0.0,0
956,2024-01-01 00:10:00,YWPS1,0,0,1,248.61772,250.00000,176.62,176.62,0.0,...,0.0,0,330.00000,365.0,250.0,365.0,250.0,0,0.0,0
957,2024-01-01 00:10:00,YWPS2,0,0,0,0.00000,0.00000,180.00,180.00,0.0,...,0.0,0,0.00000,300.0,250.0,300.0,250.0,0,0.0,0
958,2024-01-01 00:10:00,YWPS3,0,0,1,360.79999,360.00000,180.00,180.00,0.0,...,0.0,0,360.00000,360.0,280.0,360.0,280.0,0,0.0,0


In [ ]:
# Filtering the DUID, tmestamps, and power reading from the dynamic table.
# dispatchload = dispatchload[["DUID","INITIALMW","SETTLEMENTDATE"]]

In [ ]:
# Be careful about this merge, there might be data gaps so to minmise loss I used a left join.
# I think the NEM aggregate dispatch units in the Generators and Scheduled Units table, which might lead to some gaps. I haven't cleaned it because that's a headache for another day.

dispatch = pd.merge(dispatchload,NEM_gen[["DUID","Station Name",'Fuel Source - Primary','Fuel Source - Descriptor','Region']],how="left",on=["DUID"])
dispatch.sort_values(by=['DUID',"SETTLEMENTDATE"])

In [13]:
# Filtering for wind, you can add a group for solar.
# Grouping by region and timestamp then taking the sum of the 5 minute intervals.

wind_dispatch = dispatch.loc[(dispatch['Fuel Source - Primary'] == "Wind")]
wind_sum = wind_dispatch.groupby(["Region","SETTLEMENTDATE"])["INITIALMW"].sum()
wind_sum

Region  SETTLEMENTDATE     
NSW1    2021-03-01 00:05:00    366.15410
        2021-03-01 00:10:00    354.88885
        2021-03-01 00:15:00    350.23759
        2021-03-01 00:20:00    340.20724
        2021-03-01 00:25:00    329.77828
                                 ...    
VIC1    2021-03-06 23:40:00    452.19187
        2021-03-06 23:45:00    460.43404
        2021-03-06 23:50:00    473.96233
        2021-03-06 23:55:00    482.47131
        2021-03-07 00:00:00    484.76658
Name: INITIALMW, Length: 8640, dtype: float64